In [1]:
import pyspark.sql.functions as sf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window as W
from login_mysql import USER, PASSWORD, HOST, PORT, DB_NAME, URL, DRIVER

In [16]:
spark = SparkSession.builder \
        .config('spark.jars.packages','com.datastax.spark:spark-cassandra-connector_2.12:3.1.0') \
        .config('spark.jars', "C:\\spark\\mysql-connector-java-8.0.30.jar") \
        .getOrCreate() 

In [13]:
data = spark.read.format("org.apache.spark.sql.cassandra").options(table = 'tracking',keyspace = 'cycling').load()

In [15]:
data.show()

+--------------------+----+----------+-----------+----+------------+-----+--------------------+---------------+--------------------+----+--------+----+------+----+------------+--------------------+---------+--------------------+----+--------------------+-------------------+------------+-----------+----------+----------+--------+----+--------+
|         create_time| bid|        bn|campaign_id|  cd|custom_track|   de|                  dl|             dt|                  ed|  ev|group_id|  id|job_id|  md|publisher_id|                  rl|       sr|                  ts|  tz|                  ua|                uid|utm_campaign|utm_content|utm_medium|utm_source|utm_term|   v|      vp|
+--------------------+----+----------+-----------+----+------------+-----+--------------------+---------------+--------------------+----+--------+----+------+----+------------+--------------------+---------+--------------------+----+--------------------+-------------------+------------+-----------+----------+

In [17]:
sql = """(select * from events) data"""
mysql = spark.read.format('jdbc').options(url=URL, driver=DRIVER, dbtable=sql, user=USER, password=PASSWORD).load()

In [18]:
mysql.show()

+------+-------------------+-----+------------------------+---------------------+----------+----------+--------+-----------+------------+-------+------+-----------+----------+---------+-------------------+
|job_id|              dates|hours|disqualified_application|qualified_application|conversion|company_id|group_id|campaign_id|publisher_id|bid_set|clicks|impressions|spend_hour|  sources| latest_update_time|
+------+-------------------+-----+------------------------+---------------------+----------+----------+--------+-----------+------------+-------+------+-----------+----------+---------+-------------------+
|    98|2022-07-08 00:00:00|    9|                       1|                    0|         1|         1|    null|          4|           1|    2.0|   108|       null|     216.0|Cassandra|2023-07-16 10:23:15|
|    98|2022-07-13 00:00:00|   15|                       1|                    0|         1|         1|    null|          4|           1|    2.0|     1|       null|       2.0|C

In [19]:
mysql.printSchema()

root
 |-- job_id: integer (nullable = true)
 |-- dates: timestamp (nullable = true)
 |-- hours: integer (nullable = true)
 |-- disqualified_application: integer (nullable = true)
 |-- qualified_application: integer (nullable = true)
 |-- conversion: integer (nullable = true)
 |-- company_id: integer (nullable = true)
 |-- group_id: integer (nullable = true)
 |-- campaign_id: integer (nullable = true)
 |-- publisher_id: integer (nullable = true)
 |-- bid_set: double (nullable = true)
 |-- clicks: integer (nullable = true)
 |-- impressions: string (nullable = true)
 |-- spend_hour: double (nullable = true)
 |-- sources: string (nullable = true)
 |-- latest_update_time: timestamp (nullable = true)

